In [1]:
import os
import cv2
import pickle  # To store preprocessed data

In [2]:
def load_preprocessed_data(file_path):
    """Load preprocessed data from a file."""
    with open(file_path, 'rb') as file:
        return pickle.load(file)

def find_most_similar_flann(test_image_path, register_data):
    """Find the most similar image using FLANN-based matching."""
    orb = cv2.ORB_create()
    index_params = dict(algorithm=1, trees=5)  # FLANN parameters for ORB
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    test_img = cv2.imread(test_image_path, cv2.IMREAD_GRAYSCALE)
    if test_img is None:
        return None, 0

    keypoints, descriptors = orb.detectAndCompute(test_img, None)
    if descriptors is None:
        return None, 0

    # Convert test descriptors to float32
    descriptors = descriptors.astype('float32')

    best_match = None
    best_good_matches = 0

    for img_name, ref_descriptors in register_data.items():
        if ref_descriptors is not None:
            # Convert reference descriptors to float32
            ref_descriptors = ref_descriptors.astype('float32')

            # FLANN matching
            matches = flann.knnMatch(descriptors, ref_descriptors, k=2)
            # Lowe's ratio test
            good_matches = [m for m, n in matches if m.distance < 0.7 * n.distance]
            if len(good_matches) > best_good_matches:
                best_good_matches = len(good_matches)
                best_match = img_name

    return best_match, best_good_matches



In [3]:

def calculate_accuracy_flann(test_folder, preprocessed_file):
    """Calculate accuracy by comparing test images with preprocessed register data."""
    # Load preprocessed data
    register_data = load_preprocessed_data(preprocessed_file)

    # Valid image extensions
    valid_extensions = (".jpg", ".jpeg", ".png", ".bmp", ".tiff")

    correct_matches = 0
    total_images = 0

    for test_image in os.listdir(test_folder):
        test_image_path = os.path.join(test_folder, test_image)
        
        # Check if it's a valid image file
        if os.path.isfile(test_image_path) and test_image.lower().endswith(valid_extensions):
            total_images += 1
            # print("Processing image: ", test_image_path)

            # Find the most similar image
            similar_image, _ = find_most_similar_flann(test_image_path, register_data)

            if similar_image:
                # Compare the starting identifiers
                test_prefix = test_image.split("_")[0]
                similar_prefix = similar_image.split("_")[0]

                if test_prefix == similar_prefix:
                    correct_matches += 1
    
    print("Correct matches: ", correct_matches)
    print("Total images: ", total_images)
    # Calculate accuracy
    accuracy = (correct_matches / total_images) * 100 if total_images > 0 else 0
    return accuracy


In [4]:
# Main workflow
if __name__ == "__main__":
    # Paths
    test_folder = "./60_images_of_6_cows/test-images1"  # Folder containing test images
    preprocessed_file = "register_data.pkl"  # Preprocessed register database

    # Calculate accuracy
    accuracy = calculate_accuracy_flann(test_folder, preprocessed_file)
    print(f"Accuracy: {accuracy:.2f}%")

Correct matches:  40
Total images:  54
Accuracy: 74.07%
